In [1]:
import glob
import pandas as pd
import numpy as np
import re
import csv

In [2]:
input_df = pd.read_csv("data/input/eng-matching-input-v3.csv", dtype=str)
input_df.fillna('', inplace=True)
input_df["matched_voterid"] = ''

In [3]:
sos_files = glob.glob("data/input/*.txt")

frames = []

for file in sos_files:
    df = pd.read_csv(file, dtype=str)
    frames.append(df)

agg_sos_records_df = pd.concat(frames, ignore_index=True)

master_sos_df = agg_sos_records_df[['SOS_VOTERID', 'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'DATE_OF_BIRTH', 
                           'RESIDENTIAL_ADDRESS1', 'RESIDENTIAL_CITY', 'RESIDENTIAL_ZIP', 'RESIDENTIAL_STATE']]

In [4]:
sos_df = master_sos_df.fillna('')
sos_df['name'] = sos_df.apply(lambda x : x['FIRST_NAME'] + ' ' + x['LAST_NAME'], axis=1)
sos_df['birth_year'] = (pd.to_datetime(sos_df['DATE_OF_BIRTH']).dt.year).astype(int)
sos_df['row'] = ''

# standardize column names between input csv and sos voter datasets
rename_cols = {
    "RESIDENTIAL_ZIP": "zip",
    "RESIDENTIAL_ADDRESS1": "address",
    "RESIDENTIAL_CITY": "city",
    "RESIDENTIAL_STATE": "state",
    "SOS_VOTERID": "sos_voterid"
}

sos_df.rename(columns=rename_cols, inplace=True)

# mask all columns except those present in input_df
sos_df = sos_df[["row", "name", "birth_year", "address", "city", "state", "zip", "sos_voterid"]]

string_cols = ["name", "address", "city", "state"]
for col in string_cols:
    sos_df[col] = sos_df[col].apply(lambda x : x.upper())

# remove middle initials where present
frames = [sos_df,input_df]
for frame in frames:
    frame["name"] = frame["name"].apply(lambda x : re.sub(r'\s.\s', ' ', x))
    for col in string_cols:
        frame[col] = frame[col].apply(lambda x : x.upper())

merged_df = pd.concat([input_df, sos_df], ignore_index=True)

In [5]:
# remove unreasonable values for year_of_birth
# faster to cast value to int than datetime object, get same result
merged_df["birth_year"].replace({'':0}, inplace=True)
merged_df["birth_year"] = merged_df["birth_year"].astype(int)
merged_df["birth_year"] = merged_df["birth_year"].apply(lambda x : int(x) if (1920 <= x <= 2004) else 0)

# transform all strings to upper to standardize casing
string_cols = ["name", "address", "city", "state"]
for col in string_cols:
    merged_df[col] = merged_df[col].str.upper()
    
# standardize abbrev for common address suffixes
replace_add = {
    "STREET": "ST",
    "AVENUE": "AVE",
    "AV": "AVE",
    "DRIVE": "DR",
    "ROAD": "RD",
    "CIRCLE": "CR",
    "STATE ROUTE": "ST RT"
}

# From https://stackoverflow.com/a/51537735
pat = {r'\b{}\b'.format(k):v for k, v in replace_add.items()}
merged_df["address"].replace(to_replace=pat, regex=True, inplace=True)

# remove middle initials where present
merged_df["name"] = merged_df["name"].apply(lambda x : re.sub(r'\s.\s', ' ', x))

In [6]:
match_df = pd.DataFrame(columns=merged_df.columns)

# get duplicate records. All input_df rows should be (and are) present in new df
match_df = merged_df[merged_df.duplicated(subset=["birth_year","name"], keep=False) == True]
match_df = match_df.fillna('')

In [7]:
for index, x in match_df.iterrows():
    # if the row has a value for "row", it matches a row in input_df
    if x["row"] != '':
        # get its matching row (should be 1 max), add it and match to temp_df
        temp_df = match_df[(match_df["name"] == x["name"]) & (match_df["birth_year"] == x["birth_year"])].fillna('')
        # merge rows (one has value in "row", the other has value in "sos_voterid")
        temp_df = temp_df.max(numeric_only=False).to_frame().transpose()
        # copy values in "row" and "sos_voterid" columns
        row = temp_df["row"][0]
        v_id = temp_df["sos_voterid"][0]
        # find record in input_df with matching value in "row"
        idx = input_df.index[input_df["row"] == row]
        # assign value of temp_df["sos_voterid"] to input_df["matched_voterid"]
        input_df["matched_voterid"].iloc[idx] = v_id

In [8]:
input_df.to_csv("data/oh_matched_voterids.csv", index=False)